In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

In [ ]:
df = pd.read_csv("flightsCleaned.csv")
df.dtypes

In [ ]:
# Añadimos el FlightNum como variable categórica
df['FLIGHT_NUMBER']=df['FLIGHT_NUMBER'].astype(object) 

# Configuramos DATE como variable de tipo datetime
df['DATE'] = pd.to_datetime(df['DATE'])

# Mostramos el dataset
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
airports = pd.DataFrame(df.groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count())
airports = airports.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
airports["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['ORIGIN_AIRPORT'])["FLIGHT_NUMBER"].count()
airports["DELAY_PERCENTAGE"] = np.round(airports['DELAYED_FLIGHTS']/airports['TOTAL_FLIGHTS']*100,2)

# Ordenamos los aeropuertos de tal manera que los 5 con mayor % de retrasos aparezcan los primeros
airports = airports.sort_values('DELAY_PERCENTAGE',ascending=False)
airports

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df2 = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})
counties
#df

In [ ]:
{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'GEO_ID': '0500000US01001',
    'STATE': '01',
    'COUNTY': '001',
    'NAME': 'Autauga',
    'LSAD': 'County',
    'CENSUSAREA': 594.436},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-86.496774, 32.344437],
      [-86.717897, 32.402814],
      [-86.814912, 32.340803],
      [-86.890581, 32.502974],
      [-86.917595, 32.664169],
      [-86.71339, 32.661732],
      [-86.714219, 32.705694],
      [-86.413116, 32.707386],
      [-86.411172, 32.409937],
      [-86.496774, 32.344437]]]},
   'id': '01001'}]
}

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df.fips, z=df.unemp,
                                    colorscale="Viridis", zmin=0, zmax=12, marker_line_width=0))
fig.update_layout(mapbox_style="light", mapbox_accesstoken=token,
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Información Multas

In [ ]:
df.head()

Añadimos un campo con el tipo de trayecto en función de la distancia

In [ ]:
# crear una lista de nuestras condiciones
conditions = [
    (df['DISTANCE'] < 750),
    (df['DISTANCE'] >= 750) & (df['DISTANCE'] <= 1500),
    (df['DISTANCE'] > 1500)
    ]

# crear una lista de los valores que queremos asignar a cada condición
values = ['Corto', 'Medio', 'Largo']

# crear una nueva columna y utilizar np.select para asignarle valores utilizando nuestras listas como argumentos
df['TYPE'] = np.select(conditions, values)

df.head()

Hacemos lo mismo con el campo de las multas, teniendo en cuenta el coste medio del trayecto en EEUU:
"In 2015, an average flight cost $430." https://www.mercurynews.com/2016/07/08/see-how-the-cost-of-a-flight-has-changed-since-1963/

# DUDAS
¿Deberíamos usar el precio medio?
¿Deberíamos tener en cuenta ARRIVAL_DELAY o solo AIRLINE_DELAY?

In [ ]:
# crear una lista de nuestras condiciones
AVG_TICKET = 430
conditions = [
    (df['ARRIVAL_DELAY'] <= 30),
    (df['DISTANCE_TYPE']=="Corto") & (df['ARRIVAL_DELAY'] <= 60),
    (df['DISTANCE_TYPE']=="Medio") & (df['ARRIVAL_DELAY'] <= 60),
    (df['DISTANCE_TYPE']=="Largo") & (df['ARRIVAL_DELAY'] <= 60),
    (df['DISTANCE_TYPE']=="Corto") & (df['ARRIVAL_DELAY'] > 60),
    (df['DISTANCE_TYPE']=="Medio") & (df['ARRIVAL_DELAY'] > 60),
    (df['DISTANCE_TYPE']=="Largo") & (df['ARRIVAL_DELAY'] > 60)
    ]

# crear una lista de los valores que queremos asignar a cada condición
values = [0, 5000+AVG_TICKET/2, 10000+AVG_TICKET/2, 20000+AVG_TICKET/2, 7500+AVG_TICKET, 20000+AVG_TICKET, 40000+AVG_TICKET]

# crear una nueva columna y utilizar np.select para asignarle valores utilizando nuestras listas como argumentos
df['FINE'] = np.select(conditions, values)

df[27:32]

## Datos Multas

In [2]:
# Creamos la columna tipo de vuelos
condition_distance = [
    (df['DISTANCE'] < 750) ,
    (df['DISTANCE'] >= 750) & (df['DISTANCE'] <1500),
    (df['DISTANCE'] >= 1500)]

choice_distance = ['Short', 'Mid', 'Long']

# Creamos la columna tipo de retraso
condition_delay = [
    (df['ARRIVAL_DELAY'] < 0) ,
    (df['ARRIVAL_DELAY'] <= 30) ,
    (df['ARRIVAL_DELAY'] > 30) & (df['DISTANCE'] <=60),
    (df['ARRIVAL_DELAY'] > 60)]

choice_delay = ['Early arrival', '(0-30)mins', '(30-60)mins','>1h']

# Añadimos las columnas
df['TYPE'] = np.select(condition_distance, choice_distance, default='Not Specified')
df['DELAY_TYPE'] = np.select(condition_delay, choice_delay, default='Not Specified')

# Creamos la columna tipo de retraso
condition_multa = [
    (df['TYPE'] == 'Early arrival') | (df['TYPE'] == '(0-30)mins'),
    (df['DISTANCE'] == 'Short') & (df['ARRIVAL_DELAY'] == '(30-60)mins'),
    (df['DISTANCE'] == 'Mid') & (df['ARRIVAL_DELAY'] == '(30-60)mins'),
    (df['DISTANCE'] == 'Long') & (df['ARRIVAL_DELAY'] == '(30-60)mins'),   
    
    (df['DISTANCE'] == 'Short') & (df['ARRIVAL_DELAY'] == '>1h'),
    (df['DISTANCE'] == 'Mid') & (df['ARRIVAL_DELAY'] == '>1h'),
    (df['DISTANCE'] == 'Long') & (df['ARRIVAL_DELAY'] == '>1h')]

choice_multa = [0,5000,10000,20000,7500,20000,40000]

# Añadimos la columna de multas
df['FINE'] = np.select(condition_multa, choice_multa, default='Not Specified')

In [ ]:
# Variables numéricas
df["FINE"].describe()

# EDA

## Tendencia vuelos actuales

### ¿Existe variación de vuelos efectuados y retrasados mes a mes?

In [ ]:
dg = pd.DataFrame(df.groupby(df['DATE'].dt.month_name())["FLIGHT_NUMBER"].count())
dg = dg.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
dg

In [ ]:
dg["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['DATE'].dt.month_name())["FLIGHT_NUMBER"].count()
dg["PERCENTAGE"] = np.round(dg['DELAYED_FLIGHTS']/dg['TOTAL_FLIGHTS']*100,2)
ORDERED_MONTHS = ["January", "February", "March", "April", "May", "June",
      "July", "August", "September", "October", "November", "December"]
dg = dg.reindex(index = ORDERED_MONTHS)
dg.head()

In [ ]:
# Numero de vuelos totales y retrasados a lo largo del tiempo
fig = go.Figure()
fig.add_trace(go.Scatter(x=dg.index, y=dg["DELAYED_FLIGHTS"], fill='tozeroy',name="Retrasados", 
text= dg["PERCENTAGE"],hovertemplate="Porcentaje sobre el total: %{text}%")) # fill down to xaxis
fig.add_trace(go.Scatter(x=dg.index, y=dg["TOTAL_FLIGHTS"], fill='tonexty',name="Totales")) # fill to trace0 y
fig.update_layout(
    title="Variación vuelos totales y retrasados a lo largo del año",
    xaxis_title="Mes",
    yaxis_title="Vuelos",
    legend_title="Leyenda"
)
fig.show()

Como se puede observar, no hay grandes variaciones mes a mes, aunque es cierto que en febrero parecen volar bastante menos aviones.

### ¿Existe variación de vuelos efectuados y retrasados a lo largo de la semana?

In [ ]:
dh = pd.DataFrame(df.groupby(df['DATE'].dt.day_name())["FLIGHT_NUMBER"].count())
dh = dh.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
dh.head()

In [ ]:
dh["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['DATE'].dt.day_name())["FLIGHT_NUMBER"].count()
dh["PERCENTAGE"] = np.round(dh['DELAYED_FLIGHTS']/dh['TOTAL_FLIGHTS']*100,2)
dh = dh.reindex(index = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
dh.head()

In [ ]:
# Numero de vuelos totales y retrasados a lo largo del tiempo
fig = go.Figure()
fig.add_trace(go.Scatter(x=dh.index, y=dh["DELAYED_FLIGHTS"], fill='tozeroy',name="Retrasados", 
text= dh["PERCENTAGE"],hovertemplate="Vuelos retrasados : %{y}<br>" + "Porcentaje sobre el total: %{text}%")) # fill down to xaxis
fig.add_trace(go.Scatter(x=dh.index, y=dh["TOTAL_FLIGHTS"], fill='tonexty',name="Totales",
                        hovertemplate="Vuelos totales : %{y}<br>")) # fill to trace0 y
fig.update_layout(
    title="Variación vuelos totales y retrasados a lo largo del año",
    xaxis_title="Mes",
    yaxis_title="Vuelos",
    legend_title="Leyenda"
)
fig.show()

Se puede observar una notable disminución de vuelos los sábados, lo que tiene sentido puesto que la gente prefiere volar antes y aprovechar el fin de semana, no en medio de este

In [ ]:
di = pd.DataFrame(df.groupby(df['AIRLINE'])["FLIGHT_NUMBER"].count())
di = di.rename(columns={"FLIGHT_NUMBER":"TOTAL_FLIGHTS"})
di.head()

In [ ]:
di["DELAYED_FLIGHTS"] =  df[df["ARRIVAL_DELAY"]>0].groupby(df['AIRLINE'])["FLIGHT_NUMBER"].count()
di["PERCENTAGE"] = np.round(di['DELAYED_FLIGHTS']/di['TOTAL_FLIGHTS']*100,2)
di = di.sort_values(by=['TOTAL_FLIGHTS'], ascending=False)
di.head()

In [ ]:
# Numero de vuelos totales y retrasados a lo largo del tiempo
fig = go.Figure()
fig.add_trace(go.Scatter(x=di.index, y=di["DELAYED_FLIGHTS"],name="Retrasados", 
text= di["PERCENTAGE"],hovertemplate="Vuelos retrasados : %{y}<br>" + "Porcentaje sobre el total: %{text}%")) # fill down to xaxis
fig.add_trace(go.Bar(x=di.index, y=di["TOTAL_FLIGHTS"],name="Totales",
                        hovertemplate="Vuelos totales : %{y}<br>")) # fill to trace0 y
fig.update_layout(
    title="Vuelos totales y retrasados en función de la aerolinea",
    xaxis_title="Aerolinea",
    yaxis_title="Vuelos",
    legend_title="Leyenda"
)
fig.show()

In [ ]:
# Podemos bservar también la media de los retrasos de las aerolineas cada mes, para detectar aerolíneas más problemáticas y las fechas de los problemas
fig = px.density_heatmap(df, x=df["AIRLINE"], y=df["DATE"].dt.month, z='ARRIVAL_DELAY', histfunc="avg",
                         color_continuous_scale='RdYlGn_r')
fig.show()

In [ ]:
# Podemos bservar también la media de los retrasos de las aerolineas cada mes, para detectar aerolíneas más problemáticas y las fechas de los problemas
fig = px.density_heatmap(df, x=df["AIRLINE"], y=df["DATE"].dt.month, z='FINE', histfunc="avg",
                         color_continuous_scale='RdYlGn_r')
fig.show()

In [ ]:
# Market Share de las aerolíneas

No queda muy claro lo de las multas, es por pasajero? Como sabemos el número de pasajeros? Qué quieren exactamente mejorar las aerolíneas?

In [ ]:
# You can add histograms
fig = px.density_heatmap(df, x=df["DATE"].dt.month, y=df["DATE"].dt.day, z='ARRIVAL_DELAY', 
                         marginal_x="histogram", marginal_y="histogram")
fig

In [ ]:
# Correlación entre variables númericamente
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(4)

**NOTA:** cambiar colores :)

In [ ]:
# Estudio de las fechas --> ¿Algún mes en el que se produzcan más retrasos?
# Querría hacer una matriz: filas  = días del mes | columnas = meses | valor = numero de vuelos retrasados
# Plot retrasos por día del mes

In [ ]:
# Comparación razones de retraso

### Estudio origen y destino

1) mapa geográfico térmico (colo en función de salidas retrasadas)

2) rutas qué experimentan más retrasos

In [ ]:
print("Numero de vuelos repetidos: " +str(len(df['FlightNum'].unique()))+ " de un total de "+ str(df.size))

In [ ]:
# Comprobar rutas de los vuelos repetidos

### Carriers

In [ ]:
# Visualizar carriers

# Causas cese
level_count = df["AIRLINE"].value_counts()
level_count

level_df = pd.DataFrame(level_count).reset_index().rename(columns = {"index": "Carrier", "DelayedFlights": "count"})
level_count.sort_values(ascending = True,inplace = True) # Reordeno para que me quede de mayor a menor el gráfico

data = [
    go.Bar(
        y = level_count.index,
        x = level_count,
        name = "Vuelos retrasados por Compañia",
        marker_color = "cadetblue",
        width= np.repeat(0.65,len(level_count)), # Cambiar la anchura de cada barra,
        orientation = 'h'
    )
]

layout = go.Layout(title = "Vuelos retrasados por Compañia", yaxis_title = "Carriers", 
                   xaxis_title = "Número de vuelos retrasados",
                  )

fig = go.Figure(data = data, layout = layout)

fig.show()


Dentro de las compañias que sufren más retrasos, mostrar:
- rango de retraso salida
- rango de retraso llegada


### Caso anterior con aeropuertos de salida

## Cinco aeropuertos con mayor retraso

Definir si el aeropuerto con mayor retraso es en números aboslutos o comparado con la cantidad de vuelos que operan por aeropuerto --> % respecto a la cantidad de vuelos que opera

In [ ]:
# Representación de los aeropuetos
airports = pd.read_csv("airports.csv")
pd.set_option('display.max_columns', None)
airports.head()

In [ ]:
airports.columns

In [ ]:
airports['COUNTRY'].unique()

http://www.geomapik.com/desarrollo-programacion-gis/mapas-plotly-visualizacion-datos-espaciales/

In [ ]:
# Agrupar por fechas
#flights[flights["ORIGIN_AIRPORT"] == "LAX"].groupby(["YEAR","MONTH"]).size()